In [13]:
from pymongo import MongoClient
client = MongoClient()
# Initiate Database
db = client['test_database']
# Initiate Table
tab = db['test_table']

In [4]:
tab.insert({'my_key':'value'})

ObjectId('5591bda81f31104711622009')

In [7]:
tab.update({'my_key':'value'}, {'$set': {'my_key':'new_update_val'}})

{u'n': 1, u'nModified': 1, u'ok': 1, 'updatedExisting': True}

# Part 2: Practice CSS Selectors

`$('.imgWr2 > img')`

In [14]:
from bs4 import BeautifulSoup

In [23]:
with open('data/ebay_shoes.html', 'r') as data_file:
    soup = BeautifulSoup(data_file,'html.parser')

In [24]:
img_paths = [elem['src'] for elem in soup.select('.imgWr2 > img')]

In [30]:
import os.path

for img in img_paths:
    with open(os.path.join('data', img),'rb') as img_file:
        contents = img_file.read()
        out_file = open(os.path.join('data/images', os.path.basename(img)), 'w')
        out_file.write(contents)
        out_file.close()

In [31]:
import requests

In [33]:
page = requests.get('http://www.ebay.com/sch/i.html?_from=R40&_trksid=p2050601.m570.l1313.TR11.TRC1.A0.H0.Xmatcha.TRS0&_nkw=matcha&_sacat=0')

In [36]:
new_soup = BeautifulSoup(page.content)

In [40]:
title_list = [elem.text for elem in new_soup.select('.vip')]

In [41]:
title_list

[u'\r\n\t\t\t\tOrganic Matcha Green Tea Powder - 16 oz (1lb) -FREE  1-3 DAY EXPEDITED SHIPPING\n',
 u'Premium Japanese Green Tea Fine Matcha Powder 100g FREE SHIPPING',
 u'Organic Matcha Green Tea Powder - 16 oz (1lb) -  Lowest Price Anywhere',
 u'\r\n\t\t\t\tOrganic Matcha Green Tea Powder - 450g -FREE  1-3 DAY EXPEDITED SHIPPING\n',
 u'Organic Matcha Green Tea Powder - 16 oz (1lb) -FREE  1-3 DAY EXPEDITED SHIPPING',
 u'Premium Organic Japanese Green Tea Fine Matcha Powder 100g FREE SHIPPING',
 u'Matcha Powder Green Tea Pure Organic Certified Natural Premium Loose 50g',
 u'Matcha Powder Green Tea Pure Organic Certified Natural Premium Loose 100G BDU',
 u'100G Matcha Powder Green Tea Pure Organic Certified Natural Premium Loose Useful',
 u'Pure Matcha Green Tea Powder Tradition 100% Natural 8.8 Oz./250 G.',
 u'Pure Matcha Green Tea Powder Tradition 100% Natural 8.8 Oz./250 G.',
 u'Hot Matcha Powder Green Tea Pure Organic Certified Natural Premium Loose 50G New',
 u'Natural Organic Matc

#Part 3: Using a Web API to get meta data

In [42]:
key = '350b7821ccb9c1c66d1d047ad70ad7ab:10:72410915'

In [47]:
def single_query(link, payload):
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print 'WARNING', response.status_code
    else:
        return response.json()

link = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'

payload = {'api-key': '74c73309c1052e6aa1785df7cd5cef8c:9:69947183',
           'begin_date':'20150622'}

html_str = single_query(link, payload)

{u'_id': u'557ec00038f0d86829e412aa',
 u'abstract': None,
 u'blog': [],
 u'byline': {u'original': u'By PATRICK CHAPPATTE',
  u'person': [{u'firstname': u'Patrick',
    u'lastname': u'CHAPPATTE',
    u'organization': u'',
    u'rank': 1,
    u'role': u'reported'}]},
 u'document_type': u'article',
 u'headline': {u'content_kicker': u'Op-Ed Columnist',
  u'main': u'Do We Have a Plan?'},
 u'keywords': [{u'is_major': u'N',
   u'name': u'subject',
   u'rank': u'1',
   u'value': u'European Sovereign Debt Crisis (2010- )'},
  {u'is_major': u'N',
   u'name': u'glocations',
   u'rank': u'2',
   u'value': u'Greece'},
  {u'is_major': u'N',
   u'name': u'glocations',
   u'rank': u'3',
   u'value': u'Europe'},
  {u'is_major': u'N',
   u'name': u'organizations',
   u'rank': u'4',
   u'value': u'European Union'},
  {u'is_major': u'N',
   u'name': u'organizations',
   u'rank': u'5',
   u'value': u'European Central Bank'},
  {u'is_major': u'N',
   u'name': u'organizations',
   u'rank': u'6',
   u'value':

In [56]:
tab.insert(html_str['response']['docs'][0])

u'557ec00038f0d86829e412aa'

In [73]:

import time
import random
c=0

enddate='20150629'
begindate='20150622'

def date_fetch(articles):
    return "".join(articles[0]['pub_date'].split("T")[0].split("-"))
    


for i in xrange(100):
    
    # Set parameters
    payload = {'api-key': '74c73309c1052e6aa1785df7cd5cef8c:9:69947183',
               'begin_date':begindate,
              'end_date':enddate,
              'sort': 'oldest'}

    print i
    for page in xrange(100):
        payload['page'] = str(page)
        html_str = single_query(link, payload)
        time.sleep(0.1)
        tab.insert(html_str['response']['docs'])
    
    
    begindate = date_fetch(html_str['response']['docs'])



0
1


InvalidOperation: cannot do an empty bulk write

In [82]:
text_articles = tab.find({'document_type': 'article'})

In [85]:
for article in text_articles:
    u = article['web_url']
    page = requests.get(u).content
    soup = BeautifulSoup(page)
    text_content = soup.select('#story-body')[0].text
    tab.update({'_id': article['_id']}, {'$set': {'article_content':text_content}})

KeyboardInterrupt: 